In [106]:
from datasets import load_dataset
import pickle
from nltk.stem import PorterStemmer
import string
import enchant
from scipy import sparse
from math import log


In [79]:
from nltk.corpus import stopwords
english_dict = enchant.Dict("en_US")
stopwords = stopwords.words("english")

### Download all english wikipedia articles

In [2]:
dataset = load_dataset("wikipedia", "20220301.en", beam_runner="DirectRunner")

Found cached dataset wikipedia (C:/Users/k2002/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

### Download english stopwords

In [22]:
import nltk
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k2002\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k2002\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = dataset["train"]

### Keep articles containing one of those words: ["fruit", "fruits", "vegetable", "vegetables", "meat"]

In [4]:
new_dataset = []
my_words = ["fruit", "fruits", "vegetable", "vegetables", "meat"]
for site in dataset:
    tmp_text = site["text"].split(" ")
    for word in my_words:
        if word in tmp_text:
            new_dataset.append(site)
            break

In [5]:
print(len(new_dataset))

77605


In [6]:
dataset = new_dataset

### Keep articles containing one of those words: ["eat", "food"]

In [13]:
new_dataset = []
for site in dataset:
    tmp_text = site["text"].split(" ")
    if "eat" in tmp_text or "food" in tmp_text:
        new_dataset.append(site)

In [14]:
print(len(new_dataset))

21052


In [15]:
dataset = new_dataset

In [8]:
print(dataset[1454]["title"])

Tartaric acid


In [25]:
with open('dataset/my_dataset.bin', 'wb') as file:
    pickle.dump(dataset, file)

In [2]:
with open('dataset/my_dataset.bin', 'rb') as file:
    dataset = pickle.load(file)

### Reset id of every article

In [3]:
for i in range(len(dataset)):
    dataset[i]["id"] = i

In [11]:
print(dataset[1454])

{'id': 1454, 'url': 'https://en.wikipedia.org/wiki/Tartaric%20acid', 'title': 'Tartaric acid', 'text': 'Tartaric acid is a white, crystalline organic acid that occurs naturally in many fruits, most notably in grapes, but also in  bananas, tamarinds, and citrus. Its salt, potassium bitartrate, commonly known as cream of tartar, develops naturally in the process of fermentation. It is commonly mixed with sodium bicarbonate and is sold as baking powder used as a leavening agent in food preparation.  The acid itself is added to foods as an antioxidant E334 and to impart its distinctive sour taste. Naturally occurring tartaric acid is a useful raw material in organic chemical synthesis. Tartaric acid is an alpha-hydroxy-carboxylic acid, is diprotic and aldaric in acid characteristics, and is a dihydroxyl derivative of succinic acid.\n\nHistory\nTartaric acid has been known to winemakers for centuries. However, the chemical process for extraction was developed in 1769 by the Swedish chemist 

In [4]:
def simplify_string(text: str) -> list:
    def my_filter(x):
        if len(x) < 2 or any(char.isdigit() for char in x) or not english_dict.check(x):
            return False
        return True

    ps = PorterStemmer()
    text = text.lower().translate(str.maketrans("\n", " ", string.punctuation)).split(" ")
    text = [word for word in text if word not in stopwords]
    text = list(filter(my_filter, text))
    text = list(map(ps.stem, text))
    return text

In [ ]:
print(dataset[0]["text"])

In [127]:
all_words = set()
words_per_article = list()  #[id] -> dict of words
word_occurance = dict()   # word -> in_how_many_articles

In [135]:
for i in range(len(dataset)):
    text = simplify_string(dataset[i]["text"])
    word_counter = dict() # TODO better deafult_dict liblary
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1
    for word in word_counter:
        all_words.add(word)
        if word not in word_occurance:
            word_occurance[word] = 1
        else:
            word_occurance[word] += 1
    words_per_article.append(word_counter)
    if i % 100 == 0:
        print(f"SAVING for i = {i}")
        with open('dataset/all_words.bin', 'wb') as file:
            pickle.dump(all_words, file)
        with open('dataset/words_per_article.bin', 'wb') as file:
            pickle.dump(words_per_article, file)
        with open('dataset/word_occurance.bin', 'wb') as file:
            pickle.dump(word_occurance, file)
    print(f"{i} / { len(dataset)}")

4114 / 21052
4115 / 21052
4116 / 21052
4117 / 21052
4118 / 21052
4119 / 21052
4120 / 21052
4121 / 21052
4122 / 21052
4123 / 21052
4124 / 21052
4125 / 21052
4126 / 21052
4127 / 21052
4128 / 21052
4129 / 21052
4130 / 21052
4131 / 21052
4132 / 21052
4133 / 21052
4134 / 21052
4135 / 21052
4136 / 21052
4137 / 21052
4138 / 21052
4139 / 21052
4140 / 21052
4141 / 21052
4142 / 21052
4143 / 21052
4144 / 21052
4145 / 21052
4146 / 21052
4147 / 21052
4148 / 21052
4149 / 21052
4150 / 21052
4151 / 21052
4152 / 21052
4153 / 21052
4154 / 21052
4155 / 21052
4156 / 21052
4157 / 21052
4158 / 21052
4159 / 21052
4160 / 21052
4161 / 21052
4162 / 21052
4163 / 21052
4164 / 21052
4165 / 21052
4166 / 21052
4167 / 21052
4168 / 21052
4169 / 21052
4170 / 21052
4171 / 21052
4172 / 21052
4173 / 21052
4174 / 21052
4175 / 21052
4176 / 21052
4177 / 21052
4178 / 21052
4179 / 21052
4180 / 21052
4181 / 21052
4182 / 21052
4183 / 21052
4184 / 21052
4185 / 21052
4186 / 21052
4187 / 21052
4188 / 21052
4189 / 21052
4190 / 21052

In [136]:
xd = False
if xd: # anty misslick
    with open('dataset/all_words.bin', 'wb') as file:
        pickle.dump(all_words, file)
    with open('dataset/words_per_article.bin', 'wb') as file:
        pickle.dump(words_per_article, file)
    with open('dataset/word_occurance.bin', 'wb') as file:
        pickle.dump(word_occurance, file)

In [5]:
with open('dataset/all_words.bin', 'rb') as file:
    all_words = pickle.load(file)
with open('dataset/words_per_article.bin', 'rb') as file:
    words_per_article = pickle.load(file)  #[id] -> dict of words
with open('dataset/word_occurance.bin', 'rb') as file:
    word_occurance = pickle.load(file)     # word -> in_how_many_articles

In [6]:
all_words = sorted(list(all_words))
all_words_indexes = dict()
for i, word in enumerate(all_words):
    all_words_indexes[word] = i

In [7]:
N = len(dataset)
M = len(all_words)
print(f"I have {N} articles and {M} words")

I have 21052 articles and 41688 words


In [261]:
def save_sparse_mat_file(sparse_matrix, filename):
    sparse.save_npz(f"dataset/{filename}", sparse_matrix)

def create_sparse_matrix_for_all_articles(use_idf = False):
    row = []
    col = []
    word_counter = []
    for i in range(N):
        dict_of_words = words_per_article[i]
        for word in dict_of_words:
            col.append(i)
            row.append(all_words_indexes[word])
            if use_idf:  # TODO make this if nicer (and not repeted 1kkkkkk times)
                word_counter.append(dict_of_words[word] * log(N / word_occurance[word]))
            else:
                word_counter.append(dict_of_words[word])
        # print(f"{i} / {N}")
    return sparse.csr_matrix((word_counter, (row, col)), shape=(M, N))


def create_single_vector(text: str):  # already transposition (according to the script)
    text = simplify_string(text)
    tmp = []
    for word in text:
        if word in all_words:
            tmp.append(word)
    text = tmp

    word_counter = dict()
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1

    row = []
    col = []

    data = []
    for word in word_counter:
        row.append(0)
        col.append(all_words_indexes[word])
        data.append(word_counter[word])
    return sparse.csr_matrix((data, (row, col)), shape=(1, M))


def return_k_nearest_articles(searched_phrase, matrix_of_words, k = 10):
    q = create_single_vector(searched_phrase)
    # q_normalized = (normalize(q, norm="l1", axis=1))
    q_norm = sparse.linalg.norm(q, axis=1)
    ans = []
    min_val = 0
    for i in range(N):
        col = matrix_of_words.getcol(i)
        val = q * col
        if not val:
            continue
        val = (val.data / (q_norm * sparse.linalg.norm(col, axis=0)))[0]
        if len(ans) < k:
            ans.append((i, val))
        elif val > min_val:
            ans.append((i, val))
            ans.sort(key = lambda x: x[1], reverse=True)
            ans.pop(k)
            min_val = ans[-1][1]
        print(f"{i} / {N}")
    ans = [index[0] for index in ans]
    return ans

In [ ]:
k_nearest = return_k_nearest_articles("round dish with mozarella, tomato sauce and mushrooms", sparse_mat)

In [267]:
for i, index in enumerate(k_nearest):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: List of mushroom dishes | https://en.wikipedia.org/wiki/List%20of%20mushroom%20dishes
2: Tomato | https://en.wikipedia.org/wiki/Tomato
3: List of sauces | https://en.wikipedia.org/wiki/List%20of%20sauces
4: Sauce | https://en.wikipedia.org/wiki/Sauce
5: À la zingara | https://en.wikipedia.org/wiki/%C3%80%20la%20zingara
6: Stewed tomatoes | https://en.wikipedia.org/wiki/Stewed%20tomatoes
7: Edible mushroom | https://en.wikipedia.org/wiki/Edible%20mushroom
8: Rainbow sauce | https://en.wikipedia.org/wiki/Rainbow%20sauce
9: Worcestershire sauce | https://en.wikipedia.org/wiki/Worcestershire%20sauce
10: List of condiments | https://en.wikipedia.org/wiki/List%20of%20condiments


In [268]:
k_nearest_idf = return_k_nearest_articles("round dish with mozarella, tomato sauce and mushrooms", sparse_mat_idf)

1 / 21052
3 / 21052
4 / 21052
5 / 21052
6 / 21052
7 / 21052
12 / 21052
13 / 21052
14 / 21052
16 / 21052
17 / 21052
18 / 21052
20 / 21052
22 / 21052
26 / 21052
27 / 21052
28 / 21052
29 / 21052
32 / 21052
34 / 21052
36 / 21052
38 / 21052
40 / 21052
41 / 21052
43 / 21052
44 / 21052
48 / 21052
52 / 21052
55 / 21052
56 / 21052
57 / 21052
58 / 21052
60 / 21052
61 / 21052
62 / 21052
63 / 21052
64 / 21052
65 / 21052
66 / 21052
67 / 21052
68 / 21052
70 / 21052
73 / 21052
74 / 21052
75 / 21052
80 / 21052
81 / 21052
83 / 21052
86 / 21052
89 / 21052
90 / 21052
91 / 21052
92 / 21052
95 / 21052
96 / 21052
97 / 21052
102 / 21052
106 / 21052
107 / 21052
108 / 21052
109 / 21052
110 / 21052
112 / 21052
113 / 21052
114 / 21052
115 / 21052
116 / 21052
118 / 21052
120 / 21052
125 / 21052
126 / 21052
130 / 21052
131 / 21052
132 / 21052
133 / 21052
134 / 21052
135 / 21052
138 / 21052
140 / 21052
142 / 21052
143 / 21052
144 / 21052
145 / 21052
148 / 21052
149 / 21052
153 / 21052
154 / 21052
156 / 21052
158 / 

In [269]:
for i, index in enumerate(k_nearest_idf):
    print(f"{i+1}: {dataset[index]['title']} | {dataset[index]['url']}")

1: List of mushroom dishes | https://en.wikipedia.org/wiki/List%20of%20mushroom%20dishes
2: À la zingara | https://en.wikipedia.org/wiki/%C3%80%20la%20zingara
3: Sauce | https://en.wikipedia.org/wiki/Sauce
4: List of sauces | https://en.wikipedia.org/wiki/List%20of%20sauces
5: List of condiments | https://en.wikipedia.org/wiki/List%20of%20condiments
6: Tomato | https://en.wikipedia.org/wiki/Tomato
7: Worcestershire sauce | https://en.wikipedia.org/wiki/Worcestershire%20sauce
8: Stewed tomatoes | https://en.wikipedia.org/wiki/Stewed%20tomatoes
9: Picadillo | https://en.wikipedia.org/wiki/Picadillo
10: Rainbow sauce | https://en.wikipedia.org/wiki/Rainbow%20sauce


In [270]:
print("Correct answer was pizza :c")

Correct answer was pizza :c


In [10]:
xd = False
if xd:  # anty missclick
    sparse_mat = create_sparse_matrix_for_all_articles(False)
    save_sparse_mat_file(sparse_mat, "sparse_mat_default.npz")

In [12]:
xd = False
if xd:  # anty missclick
    sparse_mat_idf = create_sparse_matrix_for_all_articles(True)
    save_sparse_mat_file(sparse_mat_idf, "sparse_mat_idf.npz")

In [206]:
sparse_mat = sparse.load_npz("dataset/sparse_mat_idf.npz")
sparse_mat_idf = sparse.load_npz("dataset/sparse_mat_default.npz")

In [49]:
x = sparse_mat.getcol(0)
cx = x.tocoo()
# for i, j, v in zip(cx.row, cx.col, cx.data):
#     print(i, j, v)
#     if i > 2000:
#         break

# for i in cx.row:
#     print(x[i, 0])

4.671547117735681
7.0709191165206615
1.3532165066236812
2.5602577392546317
2.489095536339613
6.18189491004143
3.1344217265686063
3.5415926711938575
2.8703210609154213
5.124951180939916
5.54631780372361
1.107390970999496
31.821372516815376
30.108663881817243
2.792581625625665
5.327552136661375
3.2642274990394586
5.217610303579798
0.7714733940471759
4.386406342712573
3.3344448467238146
4.516671537550474
2.495411951253374
10.42195664611488
4.063106634647898
4.449419310541307
1.0495778612902873
4.3732809358953615
3.6521318707287644
1.9614685183720784
2.249938923541076
11.274525465874719
11.221130587430196
1.983319848704319
2.481681758441472
20.023963824382005
1.9005459494092627
1.5447606076591862
2.4492585717362454
1.5036759806525668
536.1539377135937
279.52863007833633
9.756349577568614
6.37123190801756
0.7862746785961857
3.7381447453888046
2.159927694294281
5.1344692808686325
6.452244466777936
1.6171629043571585
9.527897846365478
5.827616461428578
4.6506891221313715
3.583328941780909
4.0

In [7]:
def create_bow_for_string(text: str):
    text = simplify_string(text)
    word_counter = dict() # TODO better deafult_dict liblary
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1
    # bow = [0 for _ in range(len(all_words))]
    # for word in word_counter:
    #     bow[all_words.index(word)] = word_counter[word]
    # return bow